# ENSF 592 - FINAL PROJECT
## Calgary Accident Analysis
### By: Mike Lasby

# README
This notebook is the entry point of our project and functions as the View. The notebook depends on the Controller.py class to retrieve and manipulate data from the underlying data models stored as seperate .py scripts. 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from geojson import Point, MultiLineString
import geopandas as gpd
import re
import math
import folium
import matplotlib.pyplot as plt
from controller import Controller

In [2]:
ctrl = Controller()
ctrl.load_data()
ctrl.add_geo_cols()
ctrl.add_cell_col()

Loading Data...
Getting weather at yyc for month 1 in 2018
Getting weather at yyc for month 2 in 2018
Getting weather at yyc for month 3 in 2018
Getting weather at yyc for month 4 in 2018
Getting weather at yyc for month 5 in 2018
Getting weather at yyc for month 6 in 2018
Getting weather at yyc for month 7 in 2018
Getting weather at yyc for month 8 in 2018
Getting weather at yyc for month 9 in 2018
Getting weather at yyc for month 10 in 2018
Getting weather at yyc for month 11 in 2018
Getting weather at yyc for month 12 in 2018
...Data Loaded.
Adding geometry column to speeds from multiline. Flip coords? True
Adding geometry column to volumes from multilinestring. Flip coords? True
Adding geometry column to incidents from location. Flip coords? False
Adding geometry column to cameras from None. Flip coords? True
Adding geometry column to signals from Point. Flip coords? True
Adding geometry column to signs from POINT. Flip coords? True
Adding geometry column to cells from cell_bounds.

In [3]:
display(ctrl.get_frame('incidents'))

,INCIDENT INFO,DESCRIPTION,START_DT,MODIFIED_DT,QUADRANT,Longitude,Latitude,location,Count,id,date,geometry,cell
875,Southbound Deerfoot Trail and 16 Avenue NE,Multi-vehicle incident.,05/07/2018 06:30:02 PM,05/07/2018 07:07:04 PM,NE,-114.028193,51.065699,"(51.06569942197, -114.028192583328)",1,2018-05-07T17:30:0251.0656994219703-114.028192...,2018-05-07 18:30:02,"{'type': 'Point', 'coordinates': [51.065699, -...",66
2169,36 Avenue and Burnsland Road SE,Two vehicle incident.,05/25/2018 02:53:33 PM,05/25/2018 03:31:13 PM,SE,-114.059276,51.021205,"(51.021204781033, -114.059276077804)",1,2018052514533351.0212047810334-114.059276077804,2018-05-25 14:53:33,"{'type': 'Point', 'coordinates': [51.021205, -...",45
3262,Falconridge Boulevard at Castleridge Boulevard NE,Two vehicle incident.,05/25/2018 03:29:49 PM,05/25/2018 04:27:18 PM,NE,-113.958764,51.099956,"(51.099956277275, -113.958763979051)",1,2018052515294951.0999562772749-113.958763979051,2018-05-25 15:29:49,"{'type': 'Point', 'coordinates': [51.099956, -...",67
4559,Southbound Deerfoot Trail and 16 Avenue NE,Multi-vehicle incident.,05/07/2018 06:30:02 PM,05/07/2018 06:31:50 PM,NE,-114.028183,51.065739,"(51.065739428335, -114.028183476698)",1,2018-05-07T17:30:0251.0657394283348-114.028183...,2018-05-07 18:30:02,"{'type': 'Point', 'coordinates': [51.065739, -...",66
4695,Spruce Meadows Trail and Macleod Trail SE,Two vehicle incident.,05/07/2018 07:33:14 PM,05/07/2018 07:44:52 PM,SE,-114.061310,50.892629,"(50.892628633145, -114.06131023126)",1,2018-05-07T18:33:1450.8926286331446-114.061310...,2018-05-07 19:33:14,"{'type': 'Point', 'coordinates': [50.892629, -...",15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17407,Westbound 16 Avenue approaching Deerfoot Trail NE,Two vehicle incident. Blocking the left lane.,12/31/2018 05:47:41 PM,12/31/2018 05:49:08 PM,NE,-114.020057,51.067053,"(51.067052792253, -114.020056658954)",1,2018123117474151.0670527922526-114.020056658954,2018-12-31 17:47:41,"{'type': 'Point', 'coordinates': [51.067053, -...",66
17408,Southbound Crowchild Trail at Kensington Road NW,Two vehicle incident. Blocking multiple lanes,12/31/2018 05:59:28 PM,12/31/2018 06:16:01 PM,NW,-114.118501,51.052492,"(51.052492333505, -114.11850138363)",1,2018123117592851.0524923335053-114.11850138363,2018-12-31 17:59:28,"{'type': 'Point', 'coordinates': [51.052492, -...",54
17409,52 Street and 5 Avenue SE,Multi-vehicle incident.,12/31/2018 06:42:53 PM,12/31/2018 06:43:50 PM,SE,-113.956571,51.049133,"(51.049133273898, -113.956570563768)",1,2018123118425351.049133273898-113.956570563768,2018-12-31 18:42:53,"{'type': 'Point', 'coordinates': [51.049133, -...",57
17410,Eastbound Memorial Drive at 8 Street NW,Two vehicle incident.,12/31/2018 08:00:47 PM,12/31/2018 08:02:03 PM,NW,-114.079493,51.054765,"(51.05476543077, -114.079492649993)",1,2018123120004751.0547654307696-114.079492649993,2018-12-31 20:00:47,"{'type': 'Point', 'coordinates': [51.054765, -...",55


In [4]:
speeds = ctrl.get_frame('speeds')
cells = ctrl.get_frame('cells')
# display(speeds)
# display(cells)


def get_avg_speed(cell_idx):
    # print(f'testing cell {cell_idx}')
    speed_sum = 0
    num_points = 0 
    for idx, row in speeds[['cell', 'SPEED']].iterrows():
        cell_dict = row['cell']
        speed = row['SPEED']
        if cell_idx in cell_dict:
            # print(f'{cell_idx} is in {cell_dict}')
            these_points = cell_dict[cell_idx]
            # print(f'these points = {these_points}')
            speed_sum += speed*these_points
            num_points += these_points
    if num_points == 0:
        return np.nan
        # return 0
    return speed_sum/num_points

i = 0 
avg_speed = []
while i < 100: 
    avg_speed.append(get_avg_speed(i))
    i+=1
print(avg_speed)
cells['avg_speed'] = avg_speed


[nan, nan, nan, nan, nan, 59.673202614379086, 66.20689655172414, 80.44554455445545, nan, nan, nan, nan, 70.0, 70.45454545454545, 62.04255319148936, 63.6741519350215, 71.49141630901288, 67.21440397350993, 70.76843198338526, 76.34146341463415, nan, nan, nan, 72.1853388658368, 64.78494623655914, 62.06577119036591, 66.55727155727156, 69.90340620233859, 64.29906542056075, 58.07531380753138, nan, nan, nan, 60.0, 62.34993614303959, 62.788844621513945, 68.22074566260613, 70.02633889376646, 72.98299845440495, nan, nan, 73.96825396825396, 65.70743405275779, 73.4090909090909, 59.4337899543379, 58.44332175560467, 70.82397003745318, 62.31227651966627, 72.13842975206612, 80.0, nan, 60.0, 61.02028639618138, 66.2549537648613, 57.83313325330132, 44.25901201602136, 74.37106056453823, 75.69165143205363, 69.5031055900621, 80.0, 110.0, 70.859375, 63.483365949119374, 65.62115621156211, 61.7911227154047, 61.484848484848484, 67.25333333333333, 59.10411622276029, 71.12003780718337, nan, nan, 61.75438596491228,

In [5]:
display(cells.iloc[55].avg_speed)

44.25901201602136

In [6]:



def count_incidents(cell_idx):
    incidents = ctrl.get_frame('incidents')
    # print(f'testing cell {cell_idx}')
    counter = 0
    for _, cell in incidents['cell'].items(): 
        if cell_idx == cell:
            counter +=1
    return counter

i = 0 
inc_count = []

while i<100: 
    inc_count.append(count_incidents(i))
    i+=1
print(inc_count)
cells['incident_count'] = inc_count

display(cells.sort_values(by='incident_count', ascending=False))

[0, 0, 0, 0, 0, 7, 17, 13, 0, 0, 0, 0, 1, 6, 50, 111, 27, 104, 35, 2, 0, 0, 0, 9, 46, 154, 96, 93, 27, 0, 0, 0, 0, 0, 88, 225, 149, 104, 38, 0, 0, 1, 15, 69, 303, 307, 297, 63, 36, 0, 0, 3, 45, 107, 326, 465, 362, 259, 84, 5, 5, 14, 44, 123, 161, 238, 354, 290, 127, 0, 0, 26, 67, 93, 23, 161, 25, 80, 81, 0, 0, 10, 32, 84, 79, 94, 55, 63, 26, 0, 0, 0, 0, 11, 2, 5, 22, 20, 1, 0]


,cells,cell_bounds,avg_speed,incident_count
55,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.027623500000004, -114.0878505], [51.06458...",44.259012,465
56,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.027623500000004, -114.0422614], [51.06458...",74.371061,362
66,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.0645838, -114.0422614], [51.1015441, -113...",67.253333,354
54,<folium.vector_layers.Rectangle object at 0x7f...,"[[51.027623500000004, -114.1334396], [51.06458...",57.833133,326
45,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.9906632, -114.0878505], [51.0276235000000...",58.443322,307
...,...,...,...,...
39,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.953702899999996, -113.9054941], [50.99066...",NaN,0
40,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.9906632, -114.315796], [51.02762350000000...",NaN,0
49,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.9906632, -113.9054941], [51.0276235000000...",80.000000,0
1,<folium.vector_layers.Rectangle object at 0x7f...,"[[50.842822, -114.2702069], [50.8797823, -114....",NaN,0


In [7]:
df = ctrl.get_frame('temporal')
display(df)

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (C),...,Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather,date,incidents
0,-114.01,51.12,CALGARY INTL A,3031092,2018-01-01 06:00,2018,1,1,06:00,-26.6,...,NaN,89.89,NaN,NaN,NaN,-29.0,NaN,NaN,2018-01-01 06:00:00,2
1,-114.01,51.12,CALGARY INTL A,3031092,2018-01-01 07:00,2018,1,1,07:00,-22.7,...,NaN,89.88,NaN,NaN,NaN,-28.0,NaN,NaN,2018-01-01 07:00:00,0
2,-114.01,51.12,CALGARY INTL A,3031092,2018-01-01 08:00,2018,1,1,08:00,-22.3,...,NaN,89.83,NaN,NaN,NaN,-29.0,NaN,Clear,2018-01-01 08:00:00,0
3,-114.01,51.12,CALGARY INTL A,3031092,2018-01-01 09:00,2018,1,1,09:00,-19.8,...,NaN,89.81,NaN,NaN,NaN,-27.0,NaN,NaN,2018-01-01 09:00:00,3
4,-114.01,51.12,CALGARY INTL A,3031092,2018-01-01 10:00,2018,1,1,10:00,-18.0,...,NaN,89.78,NaN,NaN,NaN,-27.0,NaN,NaN,2018-01-01 10:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,-114.01,51.12,CALGARY INTL A,3031092,2018-12-31 16:00,2018,12,31,16:00,-9.0,...,NaN,89.56,NaN,NaN,NaN,-16.0,NaN,NaN,2018-12-31 16:00:00,1
8747,-114.01,51.12,CALGARY INTL A,3031092,2018-12-31 17:00,2018,12,31,17:00,-11.5,...,NaN,89.47,NaN,NaN,NaN,-18.0,NaN,Mainly Clear,2018-12-31 17:00:00,2
8748,-114.01,51.12,CALGARY INTL A,3031092,2018-12-31 18:00,2018,12,31,18:00,-11.9,...,NaN,89.34,NaN,NaN,NaN,-18.0,NaN,NaN,2018-12-31 18:00:00,1
8749,-114.01,51.12,CALGARY INTL A,3031092,2018-12-31 19:00,2018,12,31,19:00,-12.8,...,NaN,89.31,NaN,NaN,NaN,-19.0,NaN,NaN,2018-12-31 19:00:00,0


In [8]:
df =ctrl.get_frame('temporal')

df['date'] = pd.to_datetime(df['Date/Time'])
# display(df)

df2 = ctrl.get_frame('incidents')

# display(df2)
df2['date'] = pd.to_datetime(df2['START_DT'])
# display(df2)

df2['date'] = pd.to_datetime(df2['START_DT'])
mask_2018 = df2['date'].dt.year == 2018
incidents_2018 = df2[mask_2018]
# display(incidents_2018)
# display(df2['date'])


incidents_2018 = incidents_2018.resample('H', on='date')['Count'].count()
display(incidents_2018)
print(incidents_2018.name)
incidents_2018.name = 'incidents'
print(incidents_2018.name)


merged = pd.merge(df, incidents_2018, on = 'date')

# display(merged)

print(merged['Visibility (km)'].unique())

merged = merged.sort_values(by='incidents', ascending = False)
# display(merged)

grouped_weather = merged.groupby(by='Weather')

# display(grouped_weather['Count'].count())

# display(cell_groups['BLADE_TYPE'].agg('count').sort_values(ascending=False).head(10))


# cell_groups = signs_test_df.groupby(by='cell', as_index=True, sort=True)





date
2018-01-01 06:00:00    2
2018-01-01 07:00:00    0
2018-01-01 08:00:00    0
2018-01-01 09:00:00    3
2018-01-01 10:00:00    1
                      ..
2018-12-31 16:00:00    1
2018-12-31 17:00:00    2
2018-12-31 18:00:00    1
2018-12-31 19:00:00    0
2018-12-31 20:00:00    2
Freq: H, Name: Count, Length: 8751, dtype: int64

Count
incidents
[24.1 64.4 32.2 19.3 48.3  9.7  2.4  0.4  0.2  0.8  3.2  6.4  8.1  4.8
  3.6  1.6 16.1 12.9 40.2 56.3 11.3  2.   4.   1.  80.5  nan  2.8  1.2
 14.5  0.6  0.  22.5 20.9 72.4 17.7]


KeyError: 'incidents'

In [9]:
df = ctrl.get_frame('incidents')

filt = df['Count'] != 1

df.loc[filt]

,INCIDENT INFO,DESCRIPTION,START_DT,MODIFIED_DT,QUADRANT,Longitude,Latitude,location,Count,id,date,geometry,cell


In [10]:
ctrl.generate_map()

In [11]:
%%html
<iframe src="index.html" frameborder = "0" width = "960" height = "600" allowfullscreen="true" mozallowfullscreen="true" webkitallowfullscreen="true"></iframe>

In [12]:
ctrl.get_frame('incidents')

,INCIDENT INFO,DESCRIPTION,START_DT,MODIFIED_DT,QUADRANT,Longitude,Latitude,location,Count,id,date,geometry,cell
875,Southbound Deerfoot Trail and 16 Avenue NE,Multi-vehicle incident.,05/07/2018 06:30:02 PM,05/07/2018 07:07:04 PM,NE,-114.028193,51.065699,"(51.06569942197, -114.028192583328)",1,2018-05-07T17:30:0251.0656994219703-114.028192...,2018-05-07 18:30:02,"{'type': 'Point', 'coordinates': [51.065699, -...",66
2169,36 Avenue and Burnsland Road SE,Two vehicle incident.,05/25/2018 02:53:33 PM,05/25/2018 03:31:13 PM,SE,-114.059276,51.021205,"(51.021204781033, -114.059276077804)",1,2018052514533351.0212047810334-114.059276077804,2018-05-25 14:53:33,"{'type': 'Point', 'coordinates': [51.021205, -...",45
3262,Falconridge Boulevard at Castleridge Boulevard NE,Two vehicle incident.,05/25/2018 03:29:49 PM,05/25/2018 04:27:18 PM,NE,-113.958764,51.099956,"(51.099956277275, -113.958763979051)",1,2018052515294951.0999562772749-113.958763979051,2018-05-25 15:29:49,"{'type': 'Point', 'coordinates': [51.099956, -...",67
4559,Southbound Deerfoot Trail and 16 Avenue NE,Multi-vehicle incident.,05/07/2018 06:30:02 PM,05/07/2018 06:31:50 PM,NE,-114.028183,51.065739,"(51.065739428335, -114.028183476698)",1,2018-05-07T17:30:0251.0657394283348-114.028183...,2018-05-07 18:30:02,"{'type': 'Point', 'coordinates': [51.065739, -...",66
4695,Spruce Meadows Trail and Macleod Trail SE,Two vehicle incident.,05/07/2018 07:33:14 PM,05/07/2018 07:44:52 PM,SE,-114.061310,50.892629,"(50.892628633145, -114.06131023126)",1,2018-05-07T18:33:1450.8926286331446-114.061310...,2018-05-07 19:33:14,"{'type': 'Point', 'coordinates': [50.892629, -...",15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17407,Westbound 16 Avenue approaching Deerfoot Trail NE,Two vehicle incident. Blocking the left lane.,12/31/2018 05:47:41 PM,12/31/2018 05:49:08 PM,NE,-114.020057,51.067053,"(51.067052792253, -114.020056658954)",1,2018123117474151.0670527922526-114.020056658954,2018-12-31 17:47:41,"{'type': 'Point', 'coordinates': [51.067053, -...",66
17408,Southbound Crowchild Trail at Kensington Road NW,Two vehicle incident. Blocking multiple lanes,12/31/2018 05:59:28 PM,12/31/2018 06:16:01 PM,NW,-114.118501,51.052492,"(51.052492333505, -114.11850138363)",1,2018123117592851.0524923335053-114.11850138363,2018-12-31 17:59:28,"{'type': 'Point', 'coordinates': [51.052492, -...",54
17409,52 Street and 5 Avenue SE,Multi-vehicle incident.,12/31/2018 06:42:53 PM,12/31/2018 06:43:50 PM,SE,-113.956571,51.049133,"(51.049133273898, -113.956570563768)",1,2018123118425351.049133273898-113.956570563768,2018-12-31 18:42:53,"{'type': 'Point', 'coordinates': [51.049133, -...",57
17410,Eastbound Memorial Drive at 8 Street NW,Two vehicle incident.,12/31/2018 08:00:47 PM,12/31/2018 08:02:03 PM,NW,-114.079493,51.054765,"(51.05476543077, -114.079492649993)",1,2018123120004751.0547654307696-114.079492649993,2018-12-31 20:00:47,"{'type': 'Point', 'coordinates': [51.054765, -...",55
